# Population genetics

Let's do some basic population genetics with the data we already have at hand here

```
module load BCFtools R ADMIXTURE HTSlib PLINK GATK
```

We will use genomic data from all the 1000Genomes populations, in particular chromosome 18. We can define a variable with its location:

```
TGDATA="$main/share/ALL.chr18.phase3_shapeit2_mvncall_integrated_v5a.20130502.genotypes.vcf.gz" 

ls $TGDATA
```



## Heterozygosity

Now, let's look at heterozygosity at a more global scale. Since the runtime and memory requirements of this algorithm , we select a subset of variants. There are several ways of doing this, here I demonstrate a simple way with `gatk`. Let's get 25% of the SNVs in the original VCF file:

```
gatk SelectVariants --select-random-fraction 0.25 --variant $TGDATA --output random_larger_subset.vcf.gz
```

Then, again with `bcftools stats`. 

```
bcftools stats -s - random_larger_subset.vcf.gz | grep "^PSC" -B 1 > hetsTG.txt
```

If you inspect the file, you will see that there is one line per sample, with several columns containing the statistics. Column 6 is the number of Hets.

* Let's have some look at the data (in `R`)!

```
hets<-read.table("hetsTG.txt", sep="\t",header=T,comment.char="")
hist(hets[,6])
```

* Is there a relationship between homozygous alternative (1/1) and heterozygous (0/1) sites?

```
plot(hets[,5],hets[,6])

cor.test(hets[,5],hets[,6])
```

* Now, I would like to stratify this by population groups. For this, we need to load the metadata and do some merging. Unfortunately, when creating this file, the authors messed with empty columns, so we need to tell `R` to ignore this (`fill=T`). Then we merge, taking only the sample ID and heterozygous call columns.


```
meta<-read.table("appladmix/notebooks/2_popgen/data/integrated_call_samples_v3.20130502.ALL.panel", sep="\t",header=T,fill=T)

head(meta)

hetmet<-merge(hets[,c(3,6)],meta,by.x=1,by.y=1)

head(hetmet)
```

* Then, let's make a boxplot stratified by continental population (already adding colour and labels to nice it up):

```
boxplot(X.6.nHets~super_pop,data=hetmet,
    col=c("blue","green","orange","yellow","red"),
    xlab="Continental population",ylab="Heterozygous sites")
```

This very simple statistic does have a biological meaning, hence it is very informative!

* A final thing: nicely ordering the data by super-population, and plotting the distribution by more specific populations. This requires a bit of `R`-specific data handling, to get a properly sorted dataframe.

```
mypops<-c("AFR","SAS","EUR","EAS","AMR")
sortedpops<-list()
for (npop in mypops) { sortedpops[[npop]]<-unique(hetmet$pop[which(hetmet$super_pop==npop)]) }
hetmetnice<-data.frame(nHets=hetmet[,2],pop=hetmet$pop,super_pop=hetmet$super_pop)
hetmetnice$pop = factor(hetmetnice$pop, levels=unlist(sortedpops))

boxplot(nHets~pop,data=hetmetnice,
    xlab="Continental population",ylab="Heterozygous sites",
    col=c(rep("blue",length(sortedpops[[1]])),rep("green",length(sortedpops[[2]])),
    rep("orange",length(sortedpops[[3]])),rep("yellow",length(sortedpops[[4]])),
    rep("red",length(sortedpops[[5]]))))
```

Now you can see how the heterozygosities differ between super-populations. Boxplots are not an ideal way of presenting data, but convenient in base `R`.
One might rather want to get a violin plot. Here, I recommend more specialized `R` packages such as `ggplot2`. I want a violin plot with quantiles at 25\%, 50\% and 75\%. The plot should be customized with `theme()`, starting from a minimal version but with the colours I propose (`scale_fill_manual`), nice axis labels (`axis.text.x`), and some control over the title (`plot.title`), and custom or no labels on the axes (`xlab` and `ylab`). `ggtitle` provides the title content.

```
library("ggplot2")
colvec<-c("blue","green","orange","yellow","red");names(colvec)<-mypops

ggplot(hetmetnice) +
  theme_minimal() +
  geom_violin(mapping=aes(x=pop,y=nHets,fill=super_pop), draw_quantiles = c(0.25, 0.5, 0.75), scale="width" )  + 
  scale_fill_manual(values=colvec) +
  theme(
    axis.text.x = element_text(angle = 45, vjust = 1, hjust=1,size=12.5),
    plot.title = element_text(face="bold",hjust=0.5,size=15)) +
  xlab("")  + ylab("Number of heterozygous sites") + 
  ggtitle(label=paste("1000 Genomes heterozygous sites on chr18")) 
```


## Population clustering

The program [ADMIXTURE](https://dalexander.github.io/admixture/) is a widely used tool to estimate proportional ancestries from SNP data. Read more about it [here](https://dx.doi.org/10.1101/gr.094052.109). The funny thing is that his program does not actually detect admixture, but clusters SNPs based on shared occurrence across samples. The reason why they are shared can be various, including admixture, but also simply shared ancestry, bottlenecks, or other events in the history of the populations.

In any case, it is often used to understand how individuals/populations relate to each other, or more specifically how many clusters of individuals are there in the population based on their majority ancestry. For example, there might be three population clusters in bonobos, as shown in panel B of this from a [publication](https://doi.org/10.1016/j.cub.2024.09.043) on whole genome data from this species:

<img src="https://ars.els-cdn.com/content/image/1-s2.0-S0960982224012843-gr1.jpg">


This is what we can also do with the human data at hand.

Since this is quite a resource-consuming calculation (scaling with the number of SNPs), we cannot do it on the whole chromosome, so we subsample again, this time aiming for ~50,000 SNVs or 2.5% of the original SNVs (note the change in order of magnitude):

```
gatk SelectVariants --select-random-fraction 0.025 --variant $TGDATA --output random_subset.vcf.gz
```

The program does not digest a VCF file, so we have to transform the data into another file format. A suitable format is called the "plink bed" format, which we can get with the program [plink](https://www.cog-genomics.org/plink/). This tool and its various data formats are widely used in genomics, but here we simply use it to convert our data the required format, and on the go do some filtering:


```
plink2 --vcf random_subset.vcf.gz --max-alleles 2 --snps-only --make-bed --maf 0.05 --out random_subset 
```

* Let's inspect what each filtering parameter is actually doing.


Now, finally, we run ADMIXTURE, and we assume that there might be two different clusters in the data.

```
admixture random_subset.bed 2 -j1 > random_2k.out
```

A good thing is that it does not need to know how many clusters are there, as often we don't know it. So we can run it under the assumption of different numbers of clusters:

```
admixture random_subset.bed 3 -j1 > random_3k.out
admixture random_subset.bed 4 -j1 > random_4k.out
admixture random_subset.bed 5 -j1 > random_5k.out
```

That is indeed pretty easy, just apply a command line tool. We can have a look at the output.

But more important would be a visualization like the plot from the publication above.


### ADMIXTURE plots
Now, we can do this in `R` or `Rstudio`. There is a metadata table in your own directories.

```
meta<-read.table("appladmix/notebooks/2_popgen/data/integrated_call_samples_v3.20130502.ALL.panel", sep="\t",header=T,fill=T)
```

How does the data look?

```
tabl2=read.table("random_subset.2.Q")
tabl5=read.table("random_subset.5.Q")
```

Ok, so one would need to create something like a stacked barplot with these numbers.

```
barplot(t(tabl2),beside=F,col=c("green","black"),border=NA)
```


This is a bit messy, but it works. Making this a nice kind of plot is maybe a bit complicated, and I would use packages to help:

```
library("ggplot2")
library("dplyr")
library("tidyr")
library("ggpubr")
```

First, I will do some polishing with the `tidyverse` syntax to get things into a nice order. What follows will be a bit complex, but it is indeed just ordering the very same data you saw before. The goal is to separate the different global metapopulations, and display the data in a visually nice way by ordering the individuals within each population by components.

Also, this can be done in a loop to save the information for 2 to 5 clusters in an `R` object.


```
myplots=list()
for (i in 2:5) {
    tbl=read.table(paste("random_subset.",i,".Q",sep=""))
    tbl<-cbind(tbl,Label=meta[,3])

    plot_data <- tbl %>%
        mutate(id = row_number()) %>%
        gather('pop', 'prob', V1:paste0("V",i)) %>%
        group_by(id) %>%
        mutate(likely_assignment = pop[which.max(prob)],
               assingment_prob = max(prob)) %>%
        arrange(likely_assignment, desc(assingment_prob)) %>%
        ungroup() %>%
        mutate(id = forcats::fct_inorder(factor(id)))

    myplots[[i]]<-
    ggplot(plot_data, aes(id, prob, fill = pop)) +
           geom_col() +
        facet_grid(~Label, scales = 'free', space = 'free') +
        theme(
          axis.title.x = element_blank(),
          axis.text.x = element_blank(),
          axis.ticks.x = element_blank()
        ) + theme(legend.position = "none") + ggtitle(paste0("K=",i))
    }
```


Finally, you can plot everything in one figure (commented out the way to save it as pdf). Again, we use an `R` package to make it look good, basically publication-ready.



```
#pdf("figure_admixture.pdf", 10, 20)
ggarrange(myplots[[2]],myplots[[3]],myplots[[4]],myplots[[5]],ncol=1, nrow=4)
#dev.off()
```


# Principle Component Analysis

## [What is a PCA?](https://en.wikipedia.org/wiki/Principal_component_analysis)

* A statistical method to reduce dimensionality of the data.
* *E.g.* hundreds of individuals with thousands or millions of SNVs
* Many SNVs are observed across multiple individuals: co-variance
* Co-variance shows how similar individuals are - populations cluster with each other
* (unsupervised = the method does not know the labels)


### PCA in population genetics

It is a widely used summary statistic for genotype data. In many papers, it is shown as an exploratory step - we have sequenced new individuals, where do they fall in worldwide or large-scale diversity? This works for humans and many other species with sufficient amounts of data.

Here an example from [humans](https://doi.org/10.1038%2Fnature07331): 

<img src="https://media.springernature.com/full/springer-static/image/art%3A10.1038%2Fnature07331/MediaObjects/41586_2008_Article_BFnature07331_Fig1_HTML.jpg">

Or, as above, from bonobos, this time focus on panel A:

<img src="https://ars.els-cdn.com/content/image/1-s2.0-S0960982224012843-gr1.jpg">



## Getting a simple PCA

Now, there are many ways to calculate a PCA. Here, we will take one of the recently published softwares, because it is conveniently spitting out the PCA directly from a VCF file!

There are some warnings, but closer inspection tells us that it is fine, kind of.

```
mkdir pca
$main/VCF2PCACluster/bin/VCF2PCACluster -i chr21.merged.vcf.gz -o pca/geno_pca
```

That's it, and that is how a neat tool should work.

(SPOILER: most tools don't! Even here you will see some bugs that either appear on this particular computing structure, or accumulate since the program was written.)

Now we can inspect the output files as well as the plots this program created.

* We see that PC1 is separating Archaics from humans, with Europeans inbetween. Why is that the case?


## More control: doing a simple PCA in R

### Data preparation

Now, let's do a PCA with a bit more control than this out-of-the-box software. We may re-use the chr18 data from the 1000 Genomes. What we need now is a genotype matrix, or a table of numbers which can be used by a statistical method. This table should also not have missing data, or other stuff.

Fortunately, we have already subsampled the large file into a smaller file, which we can re-use again. However, it matters that here we only use SNVs with two states (one REF, one ALT, no further alleles).

A neat feature of `bcftools` is that it can also transform `vcf` formatted files into anything you need. In this case, we really just want to have the genotypes as numbers without anything else. `bcftools query` is good for that, since you can define which fields to extract, and how to separate them.

* Let's do it in one go!

```
# in BASH

bcftools view -m 2 -M2 -v snps random_subset.vcf.gz | bcftools query -f '[%GT ]\n' > allgts.txt
```

* Then we can switch into `R`.


### Data preparation in R

First, we may need the proper library to calculate this! A possible (and very nice) one is [adegenet](https://adegenet.r-forge.r-project.org/). `R` has many packages, and often they are not pre-installed on a computer/structure. If that is the case, you may need to install a package with:

```
library(adegenet)
```

Now, read the SNPS and have a look!

```
snps<-read.table("allgts.txt", sep=" ",header=F,comment.char="")
head(snps)
```

* Note that the separator is now a space, because that is how we defined it in the previous step. However, this was adding a last space to the end of the file, and R believes there is a (empty) column.

```
snps<-snps[,-2505]

table(snps[,1])
```

Now, these are not really numbers, but genotypes...

* A very easy way to turn them into single numbers would be using the `ifelse` statement like this:

```
snps[,1]<-ifelse(snps[,1]=="0|0",0,snps[,1])
snps[,1]<-ifelse(snps[,1]=="1|1",2,snps[,1])
snps[,1]<-ifelse(snps[,1]%in%c("0|1","1|0"),1,snps[,1])
table(snps[,1])
```

* However, this is not numeric, and mathematical R functions are picky about this:

```
is.numeric(snps[,1])
snps[,1]<-as.numeric(snps[,1])
is.numeric(snps[,1])
```

This looks better, but you certainly don't want to type in 2503 more times the same things, and you would very likely make mistakes on the way.

* Let's use a `for` loop and nested `ifelse` statements to make it easier! As a side-effect, the whole column will be numeric as well!

```
for (j in (2:2504)) { snps[,j]<-ifelse(snps[,j]=="0|0",0,ifelse(snps[,j]=="1|1",2,ifelse(snps[,j]%in%c("0|1","1|0"),1,NA))); print(j) }


table(snps[,3])
table(snps[,666])
is.numeric(snps[,2024])
```

I would note that at this stage it is ok to do things that way. But next level would be to write a `function` to make it more efficient...

If you feel that coming up with efficient solutions is complicated, that is fine - it is a matter of practice and experience. We need to learn things in order to do them.


### Calculating the PCA

The method we want to use is called `dudi.pca`. It needs the individuals in the rows, and the positions in the columns, that is, a transposed table to be done with `t()`. As you see, nested functions are very common in `R`, do combine things on the fly without intermediate objects. 

* Let's to this:

```
pca_object <- dudi.pca(t(snps),nf=20,scannf=F)

tb<-list()
for (j in (1:ncol(snps)))  { tb[[j]]<-table(snps[,j]) }

pca_object
summary(pca_object)
```

This works so far! We have created a higher-level `R` object with lots of information. Important are the coordinates for each individual, which are in the sub-object called `li`, which we can access with the `$` sign.

### Plotting the PCA

* Let's plot this!

```
#pdf("PCA_1000g.pdf",12,12)
plot(pca_object$li[,1],pca_object$li[,2])
#dev.off()
```

Ok, this is just some distribution of dots... It is nicer to color it. For this, we need the metadata again. In `R`, we can then create a matching colour vector:

```
meta<-read.table("appladmix/notebooks/2_popgen/data/integrated_call_samples_v3.20130502.ALL.panel", sep="\t",header=T,fill=T)
mypops<-c("AFR","SAS","EUR","EAS","AMR")
cols=c("blue","green","orange","yellow","red")

colvector<-meta$super_pop
for (pop in (1:length(mypops))) { colvector<-gsub(mypops[pop],cols[pop],colvector) }
```

* Let's plot it!

```
#pdf("PCA_1000g_col.pdf",12,12)
plot(pca_object$li[,1],pca_object$li[,2],col=colvector)
legend("top",legend=mypops, fill=cols,bty="n" )
#dev.off()
```

This is actually already quite good! We can do some more polishing to make it look nicer, and add the information how much of the variation is explained by these first two PCs (usually that is helpful to judge how informative this is in the context of population diversity).

* Let's add nice axis labels (`xlab` and `ylab`), and points instead of empty circles (with `pch`):


```
percent_variation=round(pca_object$eig/sum(pca_object$eig)*100,2)

#pdf("PCA_1000g_nice.pdf",12,12)
plot(pca_object$li[,1],pca_object$li[,2],col=colvector,pch=16,
    xlab=paste("PC1"," (",percent_variation[1],"%)",sep=""),
    ylab=paste("PC",2," (",percent_variation[2],"%)",sep=""),
    main=paste("PCA of 1000G, chr18"))
legend("topright",legend=mypops, fill=cols,bty="n" )
#dev.off()
```

Now, this looks good, and you can gain some insight into the relationships of these populations!



# Now, let's go for Challenge 1!